# Gerador e Verificador de Assinaturas Virtuais


Geradores de assinaturas digitais são ferramentas as quais garantem a autenticidade e/ou a integridade de uma dada mensagem. Para isso, utiliza-se técnicas de criptografia, normalmente criptografia assimétrica, onde uma chave privada é empregada para assinar digitalmente um dado, e o hash da mensagem é incorporado na assinatura como uma forma de validação do conteúdo.

Já verificadores de assinaturas são responsáveis por confirmar a autenticidade e/ou a integridade de uma assinatura digital. Considerando um cenário de criptografia assimétrica como o dado acima, o verificador utiliza a chave pública correspondente do emissor para decifrar a mensagem cifrada, caso a mensagem realmente seja do emissor declarado, o hash deste texto em claro deve ser correspondente ao hash enviado pelo o emissor.

Assim, neste notebook iremos implementar um gerador e verificador de assinatura digitais de documentos, utilizando técnicas avançadas em segurança da computação. Para isso, o projeto consta com quatro módulos principais: geração de chaves,  encriptação e decriptação, assinatura e verificação.


## Geração de Chaves

In [ ]:
from random import getrandbits, randrange

class KeyManager:
    p = -1
    q = -1

    e = -1

    def __init__(self) -> None:
        self.n = -1
        self.phi = -1

        self.generate()

    def generate(self):
        self.p = self.get_prime()
        self.q = self.get_prime()

        while self.p == self.q:
           self.q = self.get_prime()

        self.n = self.get_n()
        self.phi = self.get_phi()
        self.e = self.choose_e()

    def get_prime(self):
        while True:
            #cria um numero aleatorio de 1024 bits
            p = getrandbits(1024)
            #seta seu bit menos significativo para 1 para garantir ser impar
            p |= 1
            #seta seu bit mais significativo para 1 para garantir estarmos na escala correta
            p |= 1 << 1023
            #testa se esse valor e primo e apenas retorna quando um numero passar no teste
            if self.is_prime(p):
                return p

    def is_prime(self, n: int, k=128) -> bool:
        s = 0
        r = n-1
        while r & 1 == 0:
          s += 1
          r //= 2

        for _ in range(k):
          # MILLER-RABIN
          a = randrange(2, n-1)
          x = pow(a, r, n)
          if x != 1 and x != n-1:
            j = 1
            while j < s and x != n-1:
              x = pow(x, 2, n)
              if x == 1:
                return False
              j += 1
            if x != n-1:
              return False
        return True

    def choose_e(self):
      #procura um e maior do que 2 que seja coprimo a phi
      e = 2
      while e < self.phi and self.gcd(e, self.phi) != 1:
        e += 1
      return e

    def get_public_key(self):
        return (self.n, self.get_e())

    def get_private_key(self):
        return (self.n, self.get_d())

    def get_e(self):
        return self.e
    def get_d(self):
        # o proprio python  consegue calcular o inveso modular de e
        return pow(self.e, -1, self.phi)

    def get_n(self):
        return self.p * self.q

    def get_phi(self):
        return (self.p - 1) * (self.q - 1)

    def gcd(self, a, b):
        while b:
            a, b = b, a % b
        return a

In [ ]:
km = KeyManager()

pk = km.get_public_key()
sk = km.get_private_key()

print(pk)
print(sk)

(17473691584710855814048715929812957603488936939607332862685273505336597900309151263827852910527051884669632205539679244848927931125397559443355039895470656650442729671903101430824725029113921944410581856216487001138642409482892998058080305878766625054550547246624326257053819684221858655426623362310351439960192995636796085806271465883517551087345357007239813247047770167684170023496529021007972717204859436636891757525350194730242178532102858456551750662771319321483901489399528768299559600758876155017636542723173938897396920791722309487061401898770157305053146680091920243742180052337603961697229274878573160259833, 3)
(1747369158471085581404871592981295760348893693960733286268527350533659790030915126382785291052705188466963220553967924484892793112539755944335503989547065665044272967190310143082472502911392194441058185621648700113864240948289299805808030587876662505455054724662432625705381968422185865542662336231035143996019299563679608580627146588351755108734535700723981324704777016768417

In [ ]:
print(pow(3,-1,40))

27


##Encriptação e Decriptação

## Assinatura

###RSA

In [ ]:
from hashlib import sha3_256, sha3_512
class sha3:
  #funcao de hash de 256 bits
  @staticmethod
  def hash_256(message):
    if type(message) == int:
      message = str(message)
    return sha3_256(message.encode()).hexdigest()
  # funcao de hash de 512 bits
  @staticmethod
  def hash_512(message: str):
    if type(message) == int:
      message = str(message)
    return sha3_512(message.encode()).hexdigest()


In [ ]:
import base64
class rsa:
  km = KeyManager()

  def encrypt(self, message: int):
    #Recupera a chave privada
    n, d = km.get_private_key()
    #realiza a operacao (m^d)mod n
    result = pow(message, d, n)
    return str(result)


  def decrypt(self, message: int):
    #Recupera a chave publica
    n, e = km.get_public_key()
    # realiza a operacao (c^e)mod n
    result = pow(message, e, n)
    return str(result)


###OAEP

In [ ]:
from hashlib import sha3_512
class oaep:
  #usando uma seguranca de 10245 bits
  #podemos usar k0 como o mesmo tamanho da funcao de hash que usaremos em m+k1
  #Como sera usada a sha-512 colocamos k0 como 512
  k0 = 512

  #Dessa forma k1 deve ser 256 para que m + k1 + k0 = 1024
  k1 = 256

  x = None
  y = None

  def encrypt(self, message: int):
    #padding de k1 0s na mensagem
    message = message << self.k1
    #geracao de k0 bits aleatorios
    r = str(getrandbits(self.k0))

    #utilizamos o sha3-512 e fazemos um XOR entre os resultados. para obter x
    self.x = message ^ int(sha3.hash_512(r),16)

    #como X tem o mesmo tamanho de r podemos usar novamente o sha3-512 para fazer o segundo hash e obter y
    self.y = int(r) ^ int(sha3.hash_512(str(self.x)),16)

    #apos isso concatenamos os dois valores
    return str(self.x) + str(self.y)



  def decrypt(self, message: str):
    #separamos os valores x e y
    #sabemos que ambos os lados do input tem 154 digitos logo essa tarefa fica facil
    x = message[:154]
    y = message[154:]

    #primeiro recuperamos r
    r = int(y) ^ int(sha3.hash_512(x),16)
    #Com r podemos recuperar a mensagem com o padding de 0s
    v = int(x) ^ int(sha3.hash_512(str(r)),16)
    #logo apenas resta retirar esses 0s da mensagem final
    return v >> self.k1



In [ ]:
o = oaep()
message = o.encrypt(1234)
print(message)
print(o.decrypt(message))

12595320046397688107638831524266177116235769108278568378275714326068487184052026229572556376459637125729778656480051650161891716942173479422152957503806968572978066140119812938438371378855677352834709259394121391462482545161821537424917805907132574328705188068550600692780257690163036351442172923765143835029
12272876810114214767369914665860323376317810036492533392987205854063517252192


https://www.researchgate.net/publication/361293348_An_Overview_of_RSA_and_OAEP_Padding

In [ ]:
h = sha3.hash_512("123")
print(h)
h2 = sha3.hash_512(str(123))
print(h2)

48c8947f69c054a5caa934674ce8881d02bb18fb59d5a63eeaddff735b0e9801e87294783281ae49fc8287a0fd86779b27d7972d3e84f0fa0d826d7cb67dfefc
48c8947f69c054a5caa934674ce8881d02bb18fb59d5a63eeaddff735b0e9801e87294783281ae49fc8287a0fd86779b27d7972d3e84f0fa0d826d7cb67dfefc


### BASE64

In [ ]:
import base64
class base_convert:

  #converte para base 64
  @staticmethod
  def format(message: int):
    #Usa uma string de referencia
    base64_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    res = ""
    #Enquanto a mensagem nao se tornar 0 realiza o metodo de divisoes sucessivas
    while message > 0:
      remainder = message % 64
      res = base64_chars[remainder] + res
      message = message // 64
    return res


  @staticmethod
  def parse(base64_message: str):
    #Tambem usa uma string de referebncia
    base64_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    res = 0
    # procura o caracter encontrado na string e coloca e soma com o resultado multiplicado por 64
    # essa multiplicacao por 64 fara com que os termos sejam multiplicados por 64 o numero correto de vezes
    for char in base64_message:
      res = res * 64 + base64_chars.index(char)
    return res



In [ ]:
v = base_convert.format(1234)
print(v)
print(base_convert.parse(v))

TS
1234


##Verificação de Assinatura

In [ ]:
# Instanciação da classe RSA
cy = rsa()

plain_text = 1234

# Calculo do hash 256 da mensagem
h = int(sha3.hash_256(str(plain_text)), 16)

# Encryptação desse hash
e = cy.encrypt(h)

# Concatenação entre plain_text e o hash encriptado
message_to_encode = str(plain_text) + str(e)

# Conversão para a Base64
encoded_message = base_convert.format(int(message_to_encode))

print("Mensagem codificada em base64:", encoded_message)

# Parse da base64
decoded_message = str(base_convert.parse(encoded_message))

# Separação entre o plain_text e o hash encriptado
decoded_plain_text = decoded_message[:len(str(plain_text))]

encrypted_hash = decoded_message[len(str(plain_text)):]

# Decryptação do hash encriptado
decrypted_hash = cy.decrypt(int(encrypted_hash))

# Comparação entre os hashs para verificação da assinatura
print("Hash original:", h)
print("Hash decodificado:", decrypted_hash)
print("Verificação de assinatura:", int(decrypted_hash) == h)

Mensagem codificada em base64: DuryaemY6uti7m/EZKpLlmb5DYJ9QTcF4I7Cw3Vs4SYeFpLt7gDw47JecvGhYs+LNSgqgEebyIdMUN9+ooCIrOOuZwLBQEzI6quxh0gh1TRVRuquwbNlXpLAqRLtBMHwLrFpQNhfbe1c7WCfw7958YjmBfScfd7/wYrXXOLEun1ufiFbdyGZAg65HsCASEAV1N8sMzSQyFlpvvlTATkGpkgJhahJtXRSi13Pb/NIavBZq9APxh4IEZFvZ8QHLaKydTiLdASNE5AnNkqXpRJ524oy4uDv3oEEzOsilXa5ryXyp0KTBJ8ddpX47OSVunZLNw29rLP+fI1J3jbys5CgnMu2
Hash original: 13294218810688930431017767851312097404577190160289829381938228660609088934899
Hash decodificado: 13294218810688930431017767851312097404577190160289829381938228660609088934899
Verificação de assinatura: True
